In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from CosinorPy import cosinor, file_parser

## Settings

In [2]:
sns.set_style("white")
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
#flatui = ['#7fc97f','#beaed4','#fdc086','#ffff99','#386cb0','#f0027f']
sns.set_palette(colors)

# workday, weekend
workdays_colors=["#426287","#a3a3a3"]

#normal, adverse
weather_colors = ["#5f5961", "#c25050"]


In [3]:
plt.rcParams.update({'font.size': 10})

## Functions

In [4]:
def plot_df_route_models(df, df_models, colors, plot_residuals=True, plot_phase = True, folder ="", x_label = '', y_label = ''):
    for row in df_models.iterrows():
        test = row[1].test
        n_components = row[1].n_components
        period = row[1].period
        X, Y = np.array(df[df.test == test].x), np.array(df[df.test == test].y)   
        
        route_id = int(test.split("_")[1])
        
        color = colors[route_id]
        
        
        if folder:
            cosinor.fit_me(X, Y, n_components = n_components, period = period, name = test, save_to = folder+'\\'+test+'_compnts='+str(n_components) +'_per=' + str(period), plot_residuals = plot_residuals, plot_phase = plot_phase, x_label = x_label, y_label = y_label, color=color, xticks=np.arange(0,25,2,dtype=int))
        else:
            cosinor.fit_me(X, Y, n_components = n_components, period = period, name = test, save_to = "", plot_residuals = plot_residuals, plot_phase = plot_phase, x_label = x_label, y_label = y_label, color=color, xticks=np.arange(0,25,2,dtype=int))


# Basic fits

In [5]:
df = file_parser.read_excel("data\\google_traffic.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="figs\\cosinor", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results\\best_models.csv', index=False)
df_best_models.to_excel('cosinor_results\\best_models.xlsx', index=False)

In [6]:
df_best_models.columns

Index(['test', 'period', 'n_components', 'p', 'q', 'p_reject', 'q_reject',
       'RSS', 'R2', 'R2_adj', 'log-likelihood', 'amplitude', 'acrophase',
       'mesor', 'peaks', 'heights', 'troughs', 'heights2', 'ME', 'resid_SE'],
      dtype='object')

In [7]:
df_summary = df_best_models[["test", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["test"] = df_summary["test"].map(lambda x: x.split("_")[1])
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results\\best_models_summary.csv', index=False, sep="\t")

# Workdays

In [8]:
df = file_parser.read_excel("data\\google_traffic_workdays.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="figs\\cosinor_workdays", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results_workdays\\best_models_workdays.csv', index=False)
df_best_models.to_excel('cosinor_results_workdays\\best_models_workdays.xlsx', index=False)

In [9]:
df_best_models["route ID"] = df_best_models["test"].map(lambda x: x.split("_")[1])
df_best_models["day"] = df_best_models["test"].map(lambda x: x.split("_")[-1])


df_summary = df_best_models[["route ID", "day", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results_workdays\\best_models_workdays_summary.csv', index=False, sep="\t")

In [10]:
pairs = [("route_" + str(i) + "_workday", "route_" + str(i) + "_weekend") for i in range(6)] 
cosinor.plot_tuples_best_models(df, df_best_models, pairs, 
                                folder="figs\\cosinor_workdays_pairs", 
                                y_label="Pace [s/m]", 
                                colors = workdays_colors, 
                                xticks=np.arange(0,25,2,dtype=int), 
                                max_x = 24.0,
                                titles = ["Route " + str(i) for i in range(6)],
                                labels = ['Workdays', 'Weekends'])
                               
                               #['red', 'black'])

In [11]:
df_best_models.columns

Index(['test', 'period', 'n_components', 'p', 'q', 'p_reject', 'q_reject',
       'RSS', 'R2', 'R2_adj', 'log-likelihood', 'amplitude', 'acrophase',
       'mesor', 'peaks', 'heights', 'troughs', 'heights2', 'ME', 'resid_SE',
       'route ID', 'day'],
      dtype='object')

In [12]:
increase_work = {}

for work, weekends in pairs:
    route_id = int(work.split("_")[1])    
    #increase_work[route_id] = max(df_best_models.loc[df_best_models.test == work, "heights"]) / max(df_best_models.loc[df_best_models.test == weekends, "heights"])
    m1 = max(df_best_models.loc[df_best_models.test == work, "heights"].values[0])
    m2 = max(df_best_models.loc[df_best_models.test == weekends, "heights"].values[0])
    increase_work[route_id] = m1/m2

f = open('cosinor_results_workdays\\increase_work.csv', 'w')
f.write("route_id,workday/weekend\n")
for route_id, increase in increase_work.items():
    print(route_id, increase, file=f, sep=",")
f.close()
    

# Weather

In [13]:
df = file_parser.read_excel("data\\google_traffic_weather.xlsx", remove_outliers=True)
df_results = cosinor.fit_group(df, n_components = [1,2,3,4,5,6,7,8,9,10], period=24, plot=False)
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3,4,5,6,7,8,9,10])
plot_df_route_models(df, df_best_models, colors=colors, folder="figs\\cosinor_weather", plot_phase=False, y_label="Pace [s/m]")
df_best_models.to_csv('cosinor_results_weather\\best_models_weather.csv', index=False)
df_best_models.to_excel('cosinor_results_weather\\best_models_weather.xlsx', index=False)

In [14]:
df_best_models

,test,period,n_components,p,q,p_reject,q_reject,RSS,R2,R2_adj,log-likelihood,amplitude,acrophase,mesor,peaks,heights,troughs,heights2,ME,resid_SE
0,route_0_adverse_weather,24.0,8.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.051080,0.623726,0.618243,3987.854591,0.016231,2.132132,0.122434,"[7.591591591591592, 11.57957957957958, 15.8558...","[0.13492732509540276, 0.12505759905544572, 0.1...","[2.5465465465465464, 5.1891891891891895, 9.657...","[0.10620318788301147, 0.11115701701424488, 0.1...",0.013383,0.006821
1,route_0_normal_weather,24.0,9.0,1.110223e-16,1.110223e-16,0.999975,1.0,0.087783,0.563803,0.561051,10857.235505,0.011705,2.270500,0.119683,"[7.4474474474474475, 12.156156156156156, 15.32...","[0.12651163914742192, 0.12335186967043202, 0.1...","[2.4984984984984986, 5.045045045045045, 20.324...","[0.10797859823415681, 0.11196757430567426, 0.1...",0.010875,0.005546
2,route_1_adverse_weather,24.0,8.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.037681,0.635378,0.630150,4233.647733,0.012222,-3.031527,0.130595,"[7.591591591591592, 11.57957957957958, 14.7027...","[0.14121343664789515, 0.14281634770869, 0.1419...","[2.7867867867867866, 4.900900900900901, 23.639...","[0.11837266559627056, 0.12203414892658855, 0.1...",0.011401,0.005811
3,route_1_normal_weather,24.0,10.0,1.110223e-16,1.110223e-16,0.999999,1.0,0.083838,0.640492,0.638011,11121.377904,0.012177,-3.081843,0.130306,"[7.63963963963964, 11.771771771771771, 14.6066...","[0.13631956752067034, 0.1424808282478596, 0.14...","[3.3633633633633635, 20.036036036036037, 23.11...","[0.11813038847444948, 0.1301490419388097, 0.12...",0.010546,0.005379
4,route_2_adverse_weather,24.0,10.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.227998,0.531781,0.523367,3217.184269,0.027687,2.182448,0.153629,"[7.927927927927928, 10.906906906906906, 12.492...","[0.17764118908941176, 0.15681993026900456, 0.1...","[3.987987987987988, 9.705705705705705, 21.6696...","[0.12594658599166508, 0.15291522112764527, 0.1...",0.028083,0.014313
5,route_2_normal_weather,24.0,9.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.308463,0.591139,0.588575,9114.029299,0.022455,2.182448,0.150217,"[7.7357357357357355, 11.147147147147146, 15.66...","[0.1657013724461793, 0.15581439413197506, 0.17...","[1.4894894894894894, 3.6996996996997, 22.24624...","[0.13082750530120188, 0.12776205519591047, 0.1...",0.020324,0.010365
6,route_3_adverse_weather,24.0,6.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.354108,0.522903,0.517755,2939.521168,0.032044,2.044079,0.164843,"[11.915915915915916, 16.19219219219219]","[0.17313184699979545, 0.1968836732544487]","[0.43243243243243246, 4.468468468468468]","[0.13279964303190714, 0.13615339390277348]",0.035014,0.017845
7,route_3_normal_weather,24.0,7.0,1.110223e-16,1.110223e-16,1.000000,1.0,0.342010,0.684853,0.683311,8914.442070,0.023801,2.195027,0.156593,"[11.531531531531531, 15.615615615615615]","[0.17601314989272543, 0.18039365228871426]",[23.975975975975977],[0.13279157711905062],0.021438,0.010934
8,route_4_adverse_weather,24.0,7.0,1.110223e-16,1.110223e-16,0.999903,1.0,0.233510,0.538902,0.533023,3133.912849,0.025010,-2.075527,0.142901,"[7.927927927927928, 11.771771771771771, 16.096...","[0.1679040190402895, 0.15200783266901216, 0.16...","[1.5375375375375375, 4.900900900900901]","[0.11789169577650123, 0.12100431724998355]",0.028614,0.014583
9,route_4_normal_weather,24.0,9.0,1.110223e-16,1.110223e-16,0.996363,1.0,0.287914,0.611843,0.609397,9162.034509,0.019261,2.119553,0.139617,"[7.351351351351351, 12.204204204204204, 15.903...","[0.15161326250002874, 0.15078413644870123, 0.1...","[0.6726726726726726, 2.1621621621621623, 4.756...","[0.12035693094067211, 0.1211810278731694, 0.12...",0.019684,0.010039


In [15]:
df_best_models["route ID"] = df_best_models["test"].map(lambda x: x.split("_")[1])
df_best_models["weather"] = df_best_models["test"].map(lambda x: x.split("_")[2])

df_summary = df_best_models[["route ID", "weather", "n_components", "p", "q", "peaks", "heights"]].copy()
df_summary["p"] = round(df_summary["p"], 2)
df_summary["q"] = round(df_summary["q"], 2)
df_summary["peaks"] = df_summary["peaks"].map(lambda x: [round(i,2) for i in x])
df_summary["heights"] = df_summary["heights"].map(lambda x: [round(i,3) for i in x])
df_summary.to_csv('cosinor_results_weather\\best_models_weather_summary.csv', index=False, sep="\t")

In [16]:
df_summary

,route ID,weather,n_components,p,q,peaks,heights
0,0,adverse,8.0,0.0,0.0,"[7.59, 11.58, 15.86]","[0.135, 0.125, 0.139]"
1,0,normal,9.0,0.0,0.0,"[7.45, 12.16, 15.33]","[0.127, 0.123, 0.131]"
2,1,adverse,8.0,0.0,0.0,"[7.59, 11.58, 14.7, 18.02, 21.38]","[0.141, 0.143, 0.142, 0.139, 0.132]"
3,1,normal,10.0,0.0,0.0,"[7.64, 11.77, 14.61, 21.24]","[0.136, 0.142, 0.142, 0.132]"
4,2,adverse,10.0,0.0,0.0,"[7.93, 10.91, 12.49, 15.66]","[0.178, 0.157, 0.157, 0.181]"
5,2,normal,9.0,0.0,0.0,"[7.74, 11.15, 15.66]","[0.166, 0.156, 0.173]"
6,3,adverse,6.0,0.0,0.0,"[11.92, 16.19]","[0.173, 0.197]"
7,3,normal,7.0,0.0,0.0,"[11.53, 15.62]","[0.176, 0.18]"
8,4,adverse,7.0,0.0,0.0,"[7.93, 11.77, 16.1]","[0.168, 0.152, 0.163]"
9,4,normal,9.0,0.0,0.0,"[7.35, 12.2, 15.9]","[0.152, 0.151, 0.159]"


In [17]:
pairs = [("route_" + str(i) + "_normal_weather", "route_" + str(i) + "_adverse_weather") for i in range(6)] 
cosinor.plot_tuples_best_models(df, df_best_models, pairs, 
                                folder="figs\\cosinor_weather_pairs",
                                y_label="Pace [s/m]", 
                                colors = weather_colors, 
                                xticks=np.arange(0,25,2,dtype=int),
                                max_x = 24.0,
                                titles = ["Route " + str(i) for i in range(6)],
                                labels = ['Normal weather', 'Adverse weather'])#['red', 'black'])

In [18]:
increase_weather = {}

for normal, adverse in pairs:
    route_id = int(normal.split("_")[1])    
    #increase_work[route_id] = max(df_best_models.loc[df_best_models.test == work, "heights"]) / max(df_best_models.loc[df_best_models.test == weekends, "heights"])
    m1 = max(df_best_models.loc[df_best_models.test == adverse, "heights"].values[0])
    m2 = max(df_best_models.loc[df_best_models.test == normal, "heights"].values[0])
    increase_weather[route_id] = m1/m2

f = open('cosinor_results_weather\\increase_weather.csv', 'w')
f.write("route_id,adverse/normal_weather\n")
for route_id, increase in increase_weather.items():
    print(route_id, increase, file=f, sep=",")
f.close()